# IE221Kỹ thuật Lập Trình Python
## Phân tích dữ liệu thăm dò - Exploratory Data Analysis (EDA)
## (cuộc thi VinBigData Chest X-ray Abnormalities Detection)

In [ ]:
# Import thư viện Machine Learning và Data Science 
import tensorflow_probability as tfp
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import tensorflow_hub as hub
from skimage import exposure
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
import scipy

In [ ]:
# Import các thư viện Built In
from datetime import datetime
from glob import glob
import warnings
import IPython
import urllib
import zipfile
import pickle
import shutil
import string
import math
import tqdm
import time
import os
import gc
import re

In [ ]:
# Import các thư viện hỗ trợ Visualization 
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib
import plotly
import PIL
import cv2

In [ ]:
# Các PRESET thông dụng dùng trong plotly
FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", 15)]
LABEL_COLORS_WOUT_NO_FINDING = LABEL_COLORS[:8]+LABEL_COLORS[9:]

In [ ]:
# Các Import khác
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.notebook import tqdm
import pydicom

# THÔNG TIN NỀN TẢNG CỦA CUỘC THI

Trong cuộc thi này, chúng ta sẽ phát hiện những bất thường, những căn bệnh phổ biến trên phổi.
Đây là một bài toán Object Detection

<b style="text-decoration: underline; font-family: Verdana;">THÔNG TIN VỀ DATASET</b><br>

Chúng ta sẽ được cung cấp một tập dữ liệu gồm 18000 ảnh chụp X-Quang ở định dạng DICOM.
Tất cả những bức ảnh này đều đã được gán nhãn bởi các bác sĩ X-Quang đầy kinh nghiệm,có tất cả 14 căn bệnh như sau:
> **`0`** - Aortic enlargement - Phình động mạch chủ<br>
**`1`** - Atelectasis - Phù phổi<br>
**`2`** - Calcification - Vôi hóa phổi<br>
**`3`** - Cardiomegaly - Tim to<br>
**`4`** - Consolidation - Đông đặc phổi<br>
**`5`** - ILD <br>
**`6`** - Infiltration - Thâm nhiễm phỗi<br>
**`7`** - Lung Opacity - Đục phổi<br>
**`8`** - Nodule/Mass - U/Bứu<br>
**`9`** - Other lesion - Các căn bệnh khác<br>
**`10`** - Pleural effusion - Tràn dịch màng phổi<br>
**`11`** - Pleural thickening - Phổi dày<br>
**`12`** - Pneumothorax - Tràn khí phổi<br>
**`13`** - Pulmonary fibrosis - Thâm nhiễm phổi<br>
**`14`** - "No finding" Không tìm thấy căn bệnh nào trên phổi

**Lưu ý rằng trong bài toán lần này, chúng ta sẽ làm việc với ground truth từ nhiều bác sĩ cùng lúc trên một bức ảnh.
Cụ thể là tối ta sẽ có 3 bác sĩ cùng gán nhãn trên cùng một bức ảnh**

> **`train.csv`** - metadata của tập train, mỗi dòng là một object (một ảnh có thể có nhiều dòng)<br>
**`sample_submission.csv`** - một file submission mẫu - trong khuôn khổ đồ án này, chúng ta không quan tâm đến file này.<br>

<!-- <b style="text-decoration: underline; font-family: Verdana;">THÔNG TIN VỀ DATASET</b><br> -->


In [ ]:
# Định nghĩa đường dẫn đến root data directory
DATA_DIR = "/kaggle/input/vinbigdata-chest-xray-abnormalities-detection"

In [ ]:
# Đường dẫn đến training & testing dicom folders tương ứng
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

In [ ]:
# Lấy đường dẫn của tất cả ảnh trong 2 folder trên đưa vào list
TRAIN_DICOM_PATHS = [os.path.join(TRAIN_DIR, f_name) for f_name in os.listdir(TRAIN_DIR)]
TEST_DICOM_PATHS = [os.path.join(TEST_DIR, f_name) for f_name in os.listdir(TEST_DIR)]
# Sau đó thống kê số lượng mỗi tập
print(f"\n... Số lượng ảnh trong tập train: {len(TRAIN_DICOM_PATHS)} ...")
print(f"... Số lượng ảnh trong tập test: {len(TEST_DICOM_PATHS)} ...")

In [ ]:
# Định nghĩa paths đến các file .csv
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")

In [ ]:
# Đọc các file .csv và tạo các dataframe object
train_df = pd.read_csv(TRAIN_CSV)
ss_df = pd.read_csv(SS_CSV)
# Sau đó xuất ra head(10) của các df
print("\n\nTRAIN DATAFRAME\n\n")
display(train_df.head(10))
print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
display(ss_df.head(10))

In [ ]:
train_df.info()

# **CÁC HÀM TỰ ĐỊNH NGHĨA**

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    """ Chuyển một file dicom thành một numpy array 
    
    Args:
        path (str): Đường dẫn đến dicom file cần convert
        voi_lut (bool): Dùng để chuyển dữ liệu DICOM raw sang chế độ xem "human-friendly"
        fix_monochrome (bool): Chưa mô tả
        
    Returns:
        Numpy array của dicom file tương ứng
        
    """
    # Dùng tvien pydicom để đọc dicom file
    dicom = pydicom.read_file(path)
    
    # Nếu có VOI LUT (trong trường hợp available bởi thiết bị DICOM)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # Đến đoạn này ảnh XRAY có vẻ bị inverted
    # Tìm thấy cách sửa là do monochrome
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    # Normalize image array này và return
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
def plot_image(img, title="", figsize=(8,8), cmap=None):
    """ Hàm plot image để tiết kiệm thời gian """
    plt.figure(figsize=figsize)
    
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        img
        plt.imshow(img)
        
    plt.title(title, fontweight="bold")
    plt.axis(False)
    plt.show()

In [ ]:
def get_image_id(path):
    """ Hàm trả về image-id từ path """
    return path.rsplit("/", 1)[1].rsplit(".", 1)[0]

In [ ]:
def create_fractional_bbox_coordinates(row):
    """ Hàm trả về tọa độ bbox dưới dạng thập phân từ row của df """
    frac_x_min = row["x_min"]/row["img_width"]
    frac_x_max = row["x_max"]/row["img_width"]
    frac_y_min = row["y_min"]/row["img_height"]
    frac_y_max = row["y_max"]/row["img_height"]
    # Trả về định dạng [x_min, x_max, y_min, y_max]
    return frac_x_min, frac_x_max, frac_y_min, frac_y_max

In [ ]:
def draw_bboxes(img, tl, br, rgb, label="", label_location="tl", opacity=0.1, line_thickness=0):
    """ Hàm vẽ các bbox lên ảnh """
    rect = np.uint8(np.ones((br[1]-tl[1], br[0]-tl[0], 3))*rgb)
    sub_combo = cv2.addWeighted(img[tl[1]:br[1],tl[0]:br[0],:], 1-opacity, rect, opacity, 1.0)    
    img[tl[1]:br[1],tl[0]:br[0],:] = sub_combo

    if line_thickness>0:
        img = cv2.rectangle(img, tuple(tl), tuple(br), rgb, line_thickness)
        
    if label:
        # MẶC ĐỊNH
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        FONT_SCALE = 1.666
        FONT_THICKNESS = 3
        FONT_LINE_TYPE = cv2.LINE_AA
        
        if type(label)==str:
            LABEL = label.upper().replace(" ", "_")
        else:
            LABEL = f"CLASS_{label:02}"
        
        text_width, text_height = cv2.getTextSize(LABEL, FONT, FONT_SCALE, FONT_THICKNESS)[0]
        
        label_origin = {"tl":tl, "br":br, "tr":(br[0],tl[1]), "bl":(tl[0],br[1])}[label_location]
        label_offset = {
            "tl":np.array([0, -10]), "br":np.array([-text_width, text_height+10]), 
            "tr":np.array([-text_width, -10]), "bl":np.array([0, text_height+10])
        }[label_location]
        img = cv2.putText(img, LABEL, tuple(label_origin+label_offset), 
                          FONT, FONT_SCALE, rgb, FONT_THICKNESS, FONT_LINE_TYPE)
    
    return img

# **THĂM DÒ CỘT IMAGE_ID**

Cột image_id chứa một Unique IDentifier (UID) duy nhất cho biết bệnh nhân nào tương ứng với bbox nào

**THỐNG KÊ TỔNG SỐ BBOX TRÊN MỖI BỨC ẢNH**

In [ ]:
train_df.image_id

In [ ]:
train_df.image_id.value_counts()

In [ ]:
fig = px.histogram(train_df.image_id.value_counts(), 
                   log_y=False, color_discrete_sequence=['indianred'], opacity=0.7,
                   labels={"value":"Số lượng bbox trên mỗi bức ảnh"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX TRÊN MỖI BỆNH NHÂN</b>" \
                         "<i><sub>(Không Log Scale trục y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số bbox trên một bệnh nhân</b>",
                  yaxis_title="<b>Số lượng bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [ ]:
fig = px.histogram(train_df.image_id.value_counts(), 
                   log_y=True, color_discrete_sequence=['indianred'], opacity=0.7,
                   labels={"value":"Số lượng bbox trên mỗi bức ảnh"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX TRÊN MỖI BỆNH NHÂN</b>" \
                         "<i><sub>(Có Log Scale trục y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số bbox trên một bệnh nhân</b>",
                  yaxis_title="<b>Số lượng bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [ ]:
fig = px.histogram(train_df.image_id.value_counts().loc[lambda x : x>3], 
                   log_y=False, color_discrete_sequence=['indianred'], opacity=0.7,
                   labels={"value":"Số lượng bbox trên mỗi bức ảnh"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX TRÊN MỖI BỆNH NHÂN</b>" \
                         "<i><sub>(Bỏ giá trị 3 và không Log Scale trục y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số bbox trên một bệnh nhân</b>",
                  yaxis_title="<b>Số lượng bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [ ]:
scipy.stats.skew(train_df.image_id.value_counts().values)

In [ ]:
exp = train_df.image_id.value_counts().loc[lambda x : x>3]
scipy.stats.skew(exp)

**ĐẾM SỐ CĂN BỆNH TRÊN 1 BỨC ẢNH**

In [ ]:
train_df

In [ ]:
train_df.groupby('image_id')["class_id"].unique()

In [ ]:
train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(x))

In [ ]:
train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a : a != 14, x))))

In [ ]:
tmp_df = train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a: a != 14, x))))
# tmp_df.value_counts().sort_index()
fig = px.histogram(tmp_df, 
             log_y=False, color_discrete_sequence=['skyblue'], opacity=0.7,
             labels={"value":"Số lượng căn bệnh của họ"},
             title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG CĂN BỆNH CỦA BỆNH NHÂN   " \
                   "<i><sub>(KHÔNG Log Scale trục Y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số lượng căn bệnh của bệnh nhân</b>",
                  yaxis_title="<b>Số bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [ ]:
tmp_df = train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a: a != 14, x))))
# tmp_df.value_counts().sort_index()
fig = px.histogram(tmp_df, 
             log_y=True, color_discrete_sequence=['skyblue'], opacity=0.7,
             labels={"value":"Số lượng căn bệnh của họ"},
             title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG CĂN BỆNH CỦA BỆNH NHÂN   " \
                   "<i><sub>(CÓ Log Scale trục Y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số lượng căn bệnh của bệnh nhân</b>",
                  yaxis_title="<b>Số bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [ ]:
tmp_df = train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a: a != 14, x)))).loc[lambda x: x!=0]
# tmp_df.value_counts().sort_index()
fig = px.histogram(tmp_df, 
             log_y=False, color_discrete_sequence=['skyblue'], opacity=0.7,
             labels={"value":"Số lượng căn bệnh của họ"},
             title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG CĂN BỆNH CỦA BỆNH NHÂN   " \
                   "<i><sub>(LOẠI BỎ CLASS 14, KHÔNG Log Scale trục Y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số lượng căn bệnh của bệnh nhân</b>",
                  yaxis_title="<b>Số bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

# THĂM DÒ CỘT CLASS_NAME

**Đếm số bbox đối với mỗi class**

In [ ]:
fig = px.bar(train_df.class_name.value_counts(), 
             color=train_df.class_name.value_counts().index, opacity=0.85,
             color_discrete_sequence=LABEL_COLORS_WOUT_NO_FINDING,
             labels={"y":"Số bbox", "x":""},
             title="<b>PHÂN PHỐI SỐ LƯỢNG BBOX THEO CLASS</b>",)
fig.update_layout(legend_title=None,
                  font=FIG_FONT,
                  xaxis_title="",
                  yaxis_title="<b>Số bbox trên mỗi class</b>")

fig.show()

# THĂM DÒ CỘT CLASS_ID

*Bước này không có gì để phân tích. Chủ yếu là convert class_name sang class_id*

Ngoài ra còn tạo các dict int_2_str, str_2_int,str_2_clr... 

In [ ]:
# Create dictionary mappings
int_2_str = {i:train_df[train_df["class_id"]==i].iloc[0]["class_name"] for i in range(15)}
str_2_int = {v:k for k,v in int_2_str.items()}
int_2_clr = {str_2_int[k]:LABEL_COLORS[i] for i,k in enumerate(sorted(str_2_int.keys()))}

print("\n... Dictionary Mapping Class Integer to Class String Representation [int_2_str]...\n")
display(int_2_str)

print("\n... Dictionary Mapping Class String to Class Integer Representation [str_2_int]...\n")
display(str_2_int)

print("\n... Dictionary Mapping Class Integer to Color Representation [str_2_clr]...\n")
display(int_2_clr)

print("\n... Head of Train Dataframe After Dropping The Class Name Column...\n")
train_df.drop(columns=["class_name"], inplace=True)
display(train_df.head(5))

# THĂM DÒ CỘT RAD_ID 

In [ ]:
fig = px.histogram(train_df, x="rad_id", color="rad_id",opacity=0.85,
                   labels={"rad_id":"Radiologist ID"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX ĐƯỢC TẠO RA CỦA MỖI BÁC SĨ</b>",
                   ).update_xaxes(categoryorder="total descending")
fig.update_layout(legend_title="<b>RADIOLOGIST ID</b>",
                  xaxis_title="<b>Radiologist ID</b>",
                  yaxis_title="<b>Số lượng bbox được tạo ra</b>",
                  font=FIG_FONT,)
fig.show()

# Một số code thử nghiệm và code lỗi

In [ ]:
train_df.class_name.value_counts().drop("No finding")

In [ ]:
fig = px.bar(train_df.class_name.value_counts(), 
             color=train_df.class_name.value_counts().index, opacity=0.85,
             color_discrete_sequence=LABEL_COLORS_WOUT_NO_FINDING,
             labels={"y":"Annotations Per Class", "x":""},
             title="<b>Annotations Per Class</b>",)
fig.update_layout(legend_title=None,
                  font=FIG_FONT,
                  xaxis_title="",
                  yaxis_title="<b>Annotations Per Class</b>")

fig.show()

In [ ]:
fig = px.histogram(train_df.groupby('image_id')["class_name"].unique().apply(lambda x: len(x)), 
             log_y=False, color_discrete_sequence=['skyblue'], opacity=0.8,
             labels={"value":"Number of Unique Abnormalities"},
             title="<b>DISTRIBUTION OF # OF ANNOTATIONS PER PATIENT   " \
                   "<i><sub>(Log Scale for Y-Axis)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Number of Unique Abnormalities</b>",
                  yaxis_title="<b>Count of Unique Patients</b>",
                  font=FIG_FONT,)
fig.show()

.update_xaxes(categoryorder="total descending")

Cái này giúp sort